# PyTorch에서 변화도를 0으로 만들기

신경망을 구축할 때 변화도를 0으로 만들어 주는 것이 좋다. (?)

기본적으로 역전파`backward()`를 호출할 때마다 변화도가 버퍼에 쌓이기 때문이다.

## 개요

신경망을 학습시킬 때, 경사 하강법을 거쳐 모델 정확도를 높인다.

`torch.Tensor`는 PyTorch의 핵심이 되는 클래스다. 텐서를 생성할 때`requires_grad`속성을 `True`로 설정하면, 텐서의 모든 연산을 추적한다. 역전파 단계에서도 마찬가지로 변화도가 누적된다.

텐서의 변화도를 0으로 만들어 주어야 하는 경우도 있다. 학습 과정 반복문을 시작할 때, 누적되는 변화도를 정확하기 추적하기 위해서는 변화도를 0으로 만들어 주어야 한다.



In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data

import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

def get_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    elif torch.backends.mps.is_built():
        return torch.device("mps")
    else:
        return torch.device("cpu")

DEVICE = get_device()
print("DEVICE: ", DEVICE)

DEVICE:  cuda


In [2]:
## Load dataset and nomalization.

transform = transforms.Compose([
  transforms.ToTensor(),
  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(
  root='./data',
  train=True,
  download=True,
  transform=transform
)

trainloader = data.DataLoader(
  trainset,
  batch_size=4,
  shuffle=True,
  num_workers=2
)

testset = torchvision.datasets.CIFAR10(
  root='./data',
  train=False,
  download=True,
  transform=transform
)

testloader = data.DataLoader(
  testset,
  batch_size=4,
  shuffle=False,
  num_workers=2
)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


## 3. 신경망 구축하기



In [3]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    
    self.conv1 = nn.Conv2d(3, 6, 5)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fc1 = nn.Linear(16 * 5 * 5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)
    
  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 16 * 5 * 5)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x
    

## 4. 손실 함수와 옵티마이저 정의하기

분류를 위한 Cross-Entropy 손실 함수와 확률적 경사 하강법(SGD)을 사용한다.

In [4]:
import torch.nn as nn
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

## 5. 신경망을 학습시키는 과정에서 변화도를 0으로 만들기

반복적으로 신경망에 데이터를 입력하면서 최적화 한다. 이때 변화도를 0으로 만들어 주어야 한다.

In [5]:
for epoch in range(10):
  running_loss = 0.0
  for i, data in enumerate(trainloader, 0):
    inputs, labels = data
    
    optimizer.zero_grad()
    
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    
    # 통계 출력
    running_loss += loss.item()
    if i % 2000 == 1999:    # print every 2000 mini-batches
      print('[%d, %5d] loss: %.3f' %
            (epoch + 1, i + 1, running_loss / 2000))
      running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.209
[1,  4000] loss: 1.881
[1,  6000] loss: 1.704
[1,  8000] loss: 1.603
[1, 10000] loss: 1.540
[1, 12000] loss: 1.470
[2,  2000] loss: 1.410
[2,  4000] loss: 1.363
[2,  6000] loss: 1.359
[2,  8000] loss: 1.303
[2, 10000] loss: 1.309
[2, 12000] loss: 1.266
[3,  2000] loss: 1.233
[3,  4000] loss: 1.209
[3,  6000] loss: 1.201
[3,  8000] loss: 1.190
[3, 10000] loss: 1.199
[3, 12000] loss: 1.193
[4,  2000] loss: 1.101
[4,  4000] loss: 1.132
[4,  6000] loss: 1.113
[4,  8000] loss: 1.117
[4, 10000] loss: 1.122
[4, 12000] loss: 1.109
[5,  2000] loss: 1.024
[5,  4000] loss: 1.038
[5,  6000] loss: 1.031
[5,  8000] loss: 1.063
[5, 10000] loss: 1.065
[5, 12000] loss: 1.044
[6,  2000] loss: 0.977
[6,  4000] loss: 0.967
[6,  6000] loss: 0.997
[6,  8000] loss: 0.998
[6, 10000] loss: 1.013
[6, 12000] loss: 0.999
[7,  2000] loss: 0.916
[7,  4000] loss: 0.908
[7,  6000] loss: 0.947
[7,  8000] loss: 0.953
[7, 10000] loss: 0.937
[7, 12000] loss: 0.947
[8,  2000] loss: 0.843
[8,  4000] 